In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_or.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,722596,1199347308,4/28/20,1086,PPP,ERICKSON INCORPORATED,"5550 SW Macadam Ave, Suite 200",PORTLAND,OR,97239.0,...,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,NaN
1,722597,2346797200,4/16/20,1086,PPP,PACIFIC OFFICE AUTOMATION INC.,14747 NW Greenbrier Pkwy,BEAVERTON,OR,97006.0,...,999 Other,5179 Other Telecommunications,5179.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,NaN
2,722598,2507148910,4/27/21,1086,PPP,ALPHA MEDIA LLC,1211 SW 5th Ave,Portland,OR,97204.0,...,999 Other,5151 Radio and Television Broadcasting,5151.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,NaN
3,722599,4687487710,5/1/20,1086,PPP,SWANSON GROUP INC,2635 OLD HWY 99 S,ROSEBURG,OR,97471.0,...,321 Wood Products,3211 Sawmills and Wood Preservation,3211.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,NaN
4,722600,4842197707,5/1/20,1086,PPP,TIMBER PRODUCTS COMPANY LP,305 S. 4TH STREET,SPRINGFIELD,OR,97477.0,...,321 Wood Products,"3212 Veneer, Plywood, and Engineered Wood Prod...",3212.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13451,736047,9768048305,1/31/21,1086,PPS,PALEY'S INC.,1204 NW 21st Ave,Portland,OR,97209.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1204 NW 21st Ave, Portland, OR",45.531431,-122.694377,NaN
13452,736048,9833377202,4/28/20,1086,PPP,"KBH FOODS, INC.",18515 WEST UNION RD,PORTLAND,OR,97229.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"18515 WEST UNION RD, PORTLAND, OR",45.558244,-122.867883,NaN
13453,736049,9863097801,6/9/20,1086,PPP,"EISELE FARMS, INC.",8620 SE 347TH AVE,BORING,OR,97009.0,...,999 Other,1113 Fruit and Tree Nut Farming,1113.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"8620 SE 347TH AVE, BORING, OR",45.461039,-122.305030,NaN
13454,736050,9919177006,4/9/20,1086,PPP,"LAURA A. SCHWEGER, DPM, P.C.",1506 NE Willianson Blvd,BEND,OR,97701.0,...,999 Other,6211 Offices of Physicians,6211.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1506 NE Willianson Blvd, BEND, OR",44.064700,-121.269801,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
924,9284477302,"4530 Mahoney Rd NE, Saint Paul, OR",NaN,NaN
4975,4186547107,"100 C Street, Phoenix, OR",NaN,NaN
7922,8323147710,"34453 MCGARY LN, WARREN, OR",NaN,NaN
9993,2403688304,"259 North Ave, Jefferson, OR",NaN,NaN
10718,3376587104,"PO BOX 70, JEFFERSON, OR",NaN,NaN
11266,1016977100,"200 E Main St, ENTERPRISE, OR",NaN,NaN


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,1199347308,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891
1,2346797200,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285
2,2507148910,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144
3,4687487710,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436
4,4842197707,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/OR_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [27]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [28]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

<h4>FIPS Script

In [6]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [20]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
executable_path = {'executable_path': ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', **executable_path, options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('41')] #OR
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

[WDM] - Downloading: 100%|██████████| 8.84M/8.84M [00:00<00:00, 9.75MB/s]


  0%|          | 0/13450 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [21]:
geo_id_list =geo_id 

Length Check

In [10]:
print(len(geo_id_list))

22416


Matching FIPS to DataFrame

In [22]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:25062]

In [23]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
0,1199347308,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,[410510059032002]
1,2346797200,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,[410670316212001]
2,2507148910,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,[410510106022062]
3,4687487710,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,[410191300012008]
4,4842197707,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,[410390035001001]
...,...,...,...,...,...
13451,9768048305,"1204 NW 21st Ave, Portland, OR",45.531431,-122.694377,[410510049013002]
13452,9833377202,"18515 WEST UNION RD, PORTLAND, OR",45.558244,-122.867883,[410670315091021]
13453,9863097801,"8620 SE 347TH AVE, BORING, OR",45.461039,-122.305030,[410050234011005]
13454,9919177006,"1506 NE Willianson Blvd, BEND, OR",44.064700,-121.269801,[410170018012001]


Remove brackets

In [24]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
0,1199347308,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,410510059032002
1,2346797200,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,410670316212001
2,2507148910,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,410510106022062
3,4687487710,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,410191300012008
4,4842197707,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,410390035001001
...,...,...,...,...,...
13451,9768048305,"1204 NW 21st Ave, Portland, OR",45.531431,-122.694377,410510049013002
13452,9833377202,"18515 WEST UNION RD, PORTLAND, OR",45.558244,-122.867883,410670315091021
13453,9863097801,"8620 SE 347TH AVE, BORING, OR",45.461039,-122.305030,410050234011005
13454,9919177006,"1506 NE Willianson Blvd, BEND, OR",44.064700,-121.269801,410170018012001


In [25]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
1098,4436777110,"350 N. MAIN ST, LEXINGTON, OR",42.490979,-83.144072,NaN
1764,7531057707,"1 ABBEY DR, ST BENEDICT, OR",41.514392,-87.845932,NaN
4860,7689617110,"12178 GARDEN PL SW, TIGARD, OR",51.919438,19.145136,NaN
6817,7248847103,"43150 Hwy 101, PORT ORFORD, OR",39.661889,-123.481011,NaN
10694,5541307105,"34747 Hwy 101, ASTORIA, OR",41.998161,-124.208246,NaN
11737,4528678304,"2303 NW York St, 2303 NW York St, OR",43.299428,-74.217933,NaN
11787,3292878401,"1200 Highland Ave, Enterprise, OR",32.674723,-117.099477,NaN
11845,3313707106,"137 E Main Street, LOWELL, OR",42.933643,-85.341961,NaN


Parse for Merge

In [26]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,1199347308,410510059032002
1,2346797200,410670316212001
2,2507148910,410510106022062
3,4687487710,410191300012008
4,4842197707,410390035001001
...,...,...
13451,9768048305,410510049013002
13452,9833377202,410670315091021
13453,9863097801,410050234011005
13454,9919177006,410170018012001


Read in base data

In [27]:
#if from geocode file
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_or.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head()

#if from FIPS file
#fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/OR_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,722596,722596,722596,1199347308,4/28/20,1086,PPP,ERICKSON INCORPORATED,"5550 SW Macadam Ave, Suite 200",PORTLAND,...,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,NaN
1,722597,722597,722597,2346797200,4/16/20,1086,PPP,PACIFIC OFFICE AUTOMATION INC.,14747 NW Greenbrier Pkwy,BEAVERTON,...,999 Other,5179 Other Telecommunications,5179.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,NaN
2,722598,722598,722598,2507148910,4/27/21,1086,PPP,ALPHA MEDIA LLC,1211 SW 5th Ave,Portland,...,999 Other,5151 Radio and Television Broadcasting,5151.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,NaN
3,722599,722599,722599,4687487710,5/1/20,1086,PPP,SWANSON GROUP INC,2635 OLD HWY 99 S,ROSEBURG,...,321 Wood Products,3211 Sawmills and Wood Preservation,3211.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,NaN
4,722600,722600,722600,4842197707,5/1/20,1086,PPP,TIMBER PRODUCTS COMPANY LP,305 S. 4TH STREET,SPRINGFIELD,...,321 Wood Products,"3212 Veneer, Plywood, and Engineered Wood Prod...",3212.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,NaN


Merge with MainFrame

In [28]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,722596,722596,722596,1199347308,4/28/20,1086,PPP,ERICKSON INCORPORATED,"5550 SW Macadam Ave, Suite 200",PORTLAND,...,3364 Aerospace Product and Parts Manufacturing,3364.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,NaN,410510059032002
1,722597,722597,722597,2346797200,4/16/20,1086,PPP,PACIFIC OFFICE AUTOMATION INC.,14747 NW Greenbrier Pkwy,BEAVERTON,...,5179 Other Telecommunications,5179.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,NaN,410670316212001
2,722598,722598,722598,2507148910,4/27/21,1086,PPP,ALPHA MEDIA LLC,1211 SW 5th Ave,Portland,...,5151 Radio and Television Broadcasting,5151.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,NaN,410510106022062
3,722599,722599,722599,4687487710,5/1/20,1086,PPP,SWANSON GROUP INC,2635 OLD HWY 99 S,ROSEBURG,...,3211 Sawmills and Wood Preservation,3211.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,NaN,410191300012008
4,722600,722600,722600,4842197707,5/1/20,1086,PPP,TIMBER PRODUCTS COMPANY LP,305 S. 4TH STREET,SPRINGFIELD,...,"3212 Veneer, Plywood, and Engineered Wood Prod...",3212.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,NaN,410390035001001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13451,736047,736047,736047,9768048305,1/31/21,1086,PPS,PALEY'S INC.,1204 NW 21st Ave,Portland,...,7225 Restaurants and Other Eating Places,7225.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1204 NW 21st Ave, Portland, OR",45.531431,-122.694377,NaN,410510049013002
13452,736048,736048,736048,9833377202,4/28/20,1086,PPP,"KBH FOODS, INC.",18515 WEST UNION RD,PORTLAND,...,7225 Restaurants and Other Eating Places,7225.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"18515 WEST UNION RD, PORTLAND, OR",45.558244,-122.867883,NaN,410670315091021
13453,736049,736049,736049,9863097801,6/9/20,1086,PPP,"EISELE FARMS, INC.",8620 SE 347TH AVE,BORING,...,1113 Fruit and Tree Nut Farming,1113.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"8620 SE 347TH AVE, BORING, OR",45.461039,-122.305030,NaN,410050234011005
13454,736050,736050,736050,9919177006,4/9/20,1086,PPP,"LAURA A. SCHWEGER, DPM, P.C.",1506 NE Willianson Blvd,BEND,...,6211 Offices of Physicians,6211.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1506 NE Willianson Blvd, BEND, OR",44.064700,-121.269801,NaN,410170018012001


<b>Fill FIPS Columns - if FIPS file

In [29]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,722596,722596,722596,1199347308,4/28/20,1086,PPP,ERICKSON INCORPORATED,"5550 SW Macadam Ave, Suite 200",PORTLAND,...,3364.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,NaN,410510059032002,410510059032002
1,722597,722597,722597,2346797200,4/16/20,1086,PPP,PACIFIC OFFICE AUTOMATION INC.,14747 NW Greenbrier Pkwy,BEAVERTON,...,5179.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,NaN,410670316212001,410670316212001
2,722598,722598,722598,2507148910,4/27/21,1086,PPP,ALPHA MEDIA LLC,1211 SW 5th Ave,Portland,...,5151.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,NaN,410510106022062,410510106022062
3,722599,722599,722599,4687487710,5/1/20,1086,PPP,SWANSON GROUP INC,2635 OLD HWY 99 S,ROSEBURG,...,3211.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,NaN,410191300012008,410191300012008
4,722600,722600,722600,4842197707,5/1/20,1086,PPP,TIMBER PRODUCTS COMPANY LP,305 S. 4TH STREET,SPRINGFIELD,...,3212.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,NaN,410390035001001,410390035001001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13451,736047,736047,736047,9768048305,1/31/21,1086,PPS,PALEY'S INC.,1204 NW 21st Ave,Portland,...,7225.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1204 NW 21st Ave, Portland, OR",45.531431,-122.694377,NaN,410510049013002,410510049013002
13452,736048,736048,736048,9833377202,4/28/20,1086,PPP,"KBH FOODS, INC.",18515 WEST UNION RD,PORTLAND,...,7225.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"18515 WEST UNION RD, PORTLAND, OR",45.558244,-122.867883,NaN,410670315091021,410670315091021
13453,736049,736049,736049,9863097801,6/9/20,1086,PPP,"EISELE FARMS, INC.",8620 SE 347TH AVE,BORING,...,1113.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"8620 SE 347TH AVE, BORING, OR",45.461039,-122.305030,NaN,410050234011005,410050234011005
13454,736050,736050,736050,9919177006,4/9/20,1086,PPP,"LAURA A. SCHWEGER, DPM, P.C.",1506 NE Willianson Blvd,BEND,...,6211.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1506 NE Willianson Blvd, BEND, OR",44.064700,-121.269801,NaN,410170018012001,410170018012001


Drop Extra FIPS Columns

In [30]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,722596,722596,722596,1199347308,4/28/20,1086,PPP,ERICKSON INCORPORATED,"5550 SW Macadam Ave, Suite 200",PORTLAND,...,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,410510059032002
1,722597,722597,722597,2346797200,4/16/20,1086,PPP,PACIFIC OFFICE AUTOMATION INC.,14747 NW Greenbrier Pkwy,BEAVERTON,...,999 Other,5179 Other Telecommunications,5179.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,410670316212001
2,722598,722598,722598,2507148910,4/27/21,1086,PPP,ALPHA MEDIA LLC,1211 SW 5th Ave,Portland,...,999 Other,5151 Radio and Television Broadcasting,5151.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,410510106022062
3,722599,722599,722599,4687487710,5/1/20,1086,PPP,SWANSON GROUP INC,2635 OLD HWY 99 S,ROSEBURG,...,321 Wood Products,3211 Sawmills and Wood Preservation,3211.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,410191300012008
4,722600,722600,722600,4842197707,5/1/20,1086,PPP,TIMBER PRODUCTS COMPANY LP,305 S. 4TH STREET,SPRINGFIELD,...,321 Wood Products,"3212 Veneer, Plywood, and Engineered Wood Prod...",3212.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,410390035001001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13451,736047,736047,736047,9768048305,1/31/21,1086,PPS,PALEY'S INC.,1204 NW 21st Ave,Portland,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1204 NW 21st Ave, Portland, OR",45.531431,-122.694377,410510049013002
13452,736048,736048,736048,9833377202,4/28/20,1086,PPP,"KBH FOODS, INC.",18515 WEST UNION RD,PORTLAND,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"18515 WEST UNION RD, PORTLAND, OR",45.558244,-122.867883,410670315091021
13453,736049,736049,736049,9863097801,6/9/20,1086,PPP,"EISELE FARMS, INC.",8620 SE 347TH AVE,BORING,...,999 Other,1113 Fruit and Tree Nut Farming,1113.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"8620 SE 347TH AVE, BORING, OR",45.461039,-122.305030,410050234011005
13454,736050,736050,736050,9919177006,4/9/20,1086,PPP,"LAURA A. SCHWEGER, DPM, P.C.",1506 NE Willianson Blvd,BEND,...,999 Other,6211 Offices of Physicians,6211.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1506 NE Willianson Blvd, BEND, OR",44.064700,-121.269801,410170018012001


In [31]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
924,723520,723520,723520,9284477302,5/1/20,1086,PPP,"COLEMAN INDUSTIRES, INC",4530 Mahoney Rd NE,Saint Paul,...,998 All Other Miscellaneous Manufacturing,1119 Other Crop Farming,1119.0,PORTLAND DISTRICT OFFICE,c $1-2 million,100 to 249,"4530 Mahoney Rd NE, Saint Paul, OR",NaN,NaN,NaN
1098,723694,723694,723694,4436777110,4/13/20,1086,PPP,MORROW COUNTY GRAIN GROWERS,350 N. MAIN ST,LEXINGTON,...,999 Other,4931 Warehousing and Storage,4931.0,PORTLAND DISTRICT OFFICE,c $1-2 million,50 to 99,"350 N. MAIN ST, LEXINGTON, OR",42.490979,-83.144072,NaN
1764,724360,724360,724360,7531057707,5/1/20,1086,PPP,MOUNT ANGEL ABBEY,1 ABBEY DR,ST BENEDICT,...,999 Other,"6113 Colleges, Universities, and Professional ...",6113.0,PORTLAND DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"1 ABBEY DR, ST BENEDICT, OR",41.514392,-87.845932,NaN
4860,727456,727456,727456,7689617110,4/14/20,1086,PPP,COMMUNITY MAINTENANCE SERVICES INC,12178 GARDEN PL SW,TIGARD,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,PORTLAND DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"12178 GARDEN PL SW, TIGARD, OR",51.919438,19.145136,NaN
4975,727571,727571,727571,4186547107,4/13/20,1086,PPP,SUMMIT BEVERAGE OF OREGON INC,100 C Street,Phoenix,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PORTLAND DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"100 C Street, Phoenix, OR",NaN,NaN,NaN
6817,729413,729413,729413,7248847103,4/14/20,1086,PPP,"PACIFIC RIM ASSOCIATES, LLC",43150 Hwy 101,PORT ORFORD,...,998 All Other Miscellaneous Manufacturing,1133 Logging,1133.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"43150 Hwy 101, PORT ORFORD, OR",39.661889,-123.481011,NaN
7922,730518,730518,730518,8323147710,5/1/20,1086,PPP,MIDWAY VETERINARY HOSPITAL,34453 MCGARY LN,WARREN,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"34453 MCGARY LN, WARREN, OR",NaN,NaN,NaN
9993,732589,732589,732589,2403688304,1/20/21,1086,PPS,PLATINUM EXTERIORS INC,259 North Ave,Jefferson,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"259 North Ave, Jefferson, OR",NaN,NaN,NaN
10694,733290,733290,733290,5541307105,4/13/20,1086,PPP,"CLASSIC BODYWERKS, LTD",34747 Hwy 101,ASTORIA,...,999 Other,8111 Automotive Repair and Maintenance,8111.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"34747 Hwy 101, ASTORIA, OR",41.998161,-124.208246,NaN
10718,733314,733314,733314,3376587104,4/11/20,1086,PPP,"WOOD R AND D, LTD",PO BOX 70,JEFFERSON,...,999 Other,"2373 Highway, Street, and Bridge Construction",2373.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 70, JEFFERSON, OR",NaN,NaN,NaN


Convert to File

In [32]:
fips_OR = fips_merge_drop

In [33]:
fips_OR.to_csv("../../data/state_data/geo/geo_fips/OR_fips_scraped.csv") 

Review & Compare

In [34]:
geo_or_fips = pd.read_csv("../../data/state_data/geo/geo_fips/OR_fips_scraped.csv") 
geo_or_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,722596,722596,722596,1199347308,4/28/20,1086,PPP,ERICKSON INCORPORATED,"5550 SW Macadam Ave, Suite 200",...,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,4.105101e+14
1,1,722597,722597,722597,2346797200,4/16/20,1086,PPP,PACIFIC OFFICE AUTOMATION INC.,14747 NW Greenbrier Pkwy,...,999 Other,5179 Other Telecommunications,5179.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,4.106703e+14
2,2,722598,722598,722598,2507148910,4/27/21,1086,PPP,ALPHA MEDIA LLC,1211 SW 5th Ave,...,999 Other,5151 Radio and Television Broadcasting,5151.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,4.105101e+14
3,3,722599,722599,722599,4687487710,5/1/20,1086,PPP,SWANSON GROUP INC,2635 OLD HWY 99 S,...,321 Wood Products,3211 Sawmills and Wood Preservation,3211.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,4.101913e+14
4,4,722600,722600,722600,4842197707,5/1/20,1086,PPP,TIMBER PRODUCTS COMPANY LP,305 S. 4TH STREET,...,321 Wood Products,"3212 Veneer, Plywood, and Engineered Wood Prod...",3212.0,PORTLAND DISTRICT OFFICE,e $5-10 million,500 or more,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,4.103900e+14


In [35]:
##### fill in missing Lat, Long, FIPS
geo_or_fips.loc[geo_or_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
924,924,723520,723520,723520,9284477302,5/1/20,1086,PPP,"COLEMAN INDUSTIRES, INC",4530 Mahoney Rd NE,...,998 All Other Miscellaneous Manufacturing,1119 Other Crop Farming,1119.0,PORTLAND DISTRICT OFFICE,c $1-2 million,100 to 249,"4530 Mahoney Rd NE, Saint Paul, OR",NaN,NaN,NaN
1098,1098,723694,723694,723694,4436777110,4/13/20,1086,PPP,MORROW COUNTY GRAIN GROWERS,350 N. MAIN ST,...,999 Other,4931 Warehousing and Storage,4931.0,PORTLAND DISTRICT OFFICE,c $1-2 million,50 to 99,"350 N. MAIN ST, LEXINGTON, OR",42.490979,-83.144072,NaN
1764,1764,724360,724360,724360,7531057707,5/1/20,1086,PPP,MOUNT ANGEL ABBEY,1 ABBEY DR,...,999 Other,"6113 Colleges, Universities, and Professional ...",6113.0,PORTLAND DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"1 ABBEY DR, ST BENEDICT, OR",41.514392,-87.845932,NaN
4860,4860,727456,727456,727456,7689617110,4/14/20,1086,PPP,COMMUNITY MAINTENANCE SERVICES INC,12178 GARDEN PL SW,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,PORTLAND DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"12178 GARDEN PL SW, TIGARD, OR",51.919438,19.145136,NaN
4975,4975,727571,727571,727571,4186547107,4/13/20,1086,PPP,SUMMIT BEVERAGE OF OREGON INC,100 C Street,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PORTLAND DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"100 C Street, Phoenix, OR",NaN,NaN,NaN
6817,6817,729413,729413,729413,7248847103,4/14/20,1086,PPP,"PACIFIC RIM ASSOCIATES, LLC",43150 Hwy 101,...,998 All Other Miscellaneous Manufacturing,1133 Logging,1133.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"43150 Hwy 101, PORT ORFORD, OR",39.661889,-123.481011,NaN
7922,7922,730518,730518,730518,8323147710,5/1/20,1086,PPP,MIDWAY VETERINARY HOSPITAL,34453 MCGARY LN,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"34453 MCGARY LN, WARREN, OR",NaN,NaN,NaN
9993,9993,732589,732589,732589,2403688304,1/20/21,1086,PPS,PLATINUM EXTERIORS INC,259 North Ave,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"259 North Ave, Jefferson, OR",NaN,NaN,NaN
10694,10694,733290,733290,733290,5541307105,4/13/20,1086,PPP,"CLASSIC BODYWERKS, LTD",34747 Hwy 101,...,999 Other,8111 Automotive Repair and Maintenance,8111.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"34747 Hwy 101, ASTORIA, OR",41.998161,-124.208246,NaN
10718,10718,733314,733314,733314,3376587104,4/11/20,1086,PPP,"WOOD R AND D, LTD",PO BOX 70,...,999 Other,"2373 Highway, Street, and Bridge Construction",2373.0,PORTLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 70, JEFFERSON, OR",NaN,NaN,NaN


In [39]:
geo_or_fips["BusinessType"] = geo_or_fips["BusinessType"].fillna('Not Available')
geo_or_fips["BusinessType"].unique()
geo_or_fips.to_csv("../../data/state_data/geo/geo_fips/OR_fips_scraped.csv") 
geo_or_fips =pd.read_csv("../../data/state_data/geo/geo_fips/OR_fips_scraped.csv") 
geo_or_fips["BusinessType"].unique()

array(['Limited Liability Company(LLC)', 'Subchapter S Corporation',
       'Corporation', 'Partnership', 'Limited Liability Partnership',
       '501(c)3 – Non Profit', 'Employee Stock Ownership Plan(ESOP)',
       'Professional Association', 'Sole Proprietorship',
       'Non-Profit Organization', 'Not Available', 'Joint Venture',
       'Cooperative', '501(c)6 – Non Profit Membership',
       'Self-Employed Individuals', 'Trust', 'Independent Contractors',
       'Non-Profit Childcare Center', 'Single Member LLC',
       'Tenant in Common', 'Qualified Joint-Venture (spouses)'],
      dtype=object)